In [ ]:
import cv2
import mediapipe as mp
import numpy as np # Needed for distance calculation in pinch gesture

In [ ]:


# --- 1. Initialization and Setup ---
mp_hands = mp.solutions.hands 
mp_drawing = mp.solutions.drawing_utils 

# Landmark IDs for Tips (4, 8, 12, 16, 20)
TIP_IDS = [4, 8, 12, 16, 20] 

cap = cv2.VideoCapture(0)

# --- Function to Determine which Fingers are Up ---
def check_fingers_up(hand_landmarks):
    """
    Checks which fingers (Thumb, Index, Middle, Ring, Pinky) are extended.
    Returns a list of booleans: [is_thumb_up, is_index_up, ..., is_pinky_up]
    """
    fingers_up = []
    
    # Get all 21 normalized landmarks
    landmarks = hand_landmarks.landmark
    
    # 1. Thumb Check (Landmark 4 vs 3 - X-axis movement)
    # The thumb is up if the X-coordinate of the Tip (4) is greater than the X-coordinate of the DIP (3)
    # Note: Logic needs to consider right/left hand orientation later, but this works for a facing hand.
    if landmarks[TIP_IDS[0]].x > landmarks[TIP_IDS[0] - 1].x: # Check Tip (4) vs DIP (3)
        fingers_up.append(True)
    else:
        fingers_up.append(False)
        
    # 2. Check other 4 Fingers (Index, Middle, Ring, Pinky) - Y-axis movement
    # For fingers, the tip's Y-coordinate must be smaller than the second knuckle (PIP) Y-coordinate to be "up" (since Y-origin is top-left)
    # Tip (8, 12, 16, 20) vs PIP (6, 10, 14, 18)
    for i in range(1, 5):
        tip_y = landmarks[TIP_IDS[i]].y
        pip_y = landmarks[TIP_IDS[i] - 2].y # PIP is 2 landmarks before the tip
        
        if tip_y < pip_y:
            fingers_up.append(True)
        else:
            fingers_up.append(False)
            
    return fingers_up

# --- Function to Detect Pinch Gesture ---
def detect_pinch(hand_landmarks, h, w):
    """
    Detects if the thumb tip and index finger tip are close enough for a pinch.
    h, w are image dimensions for pixel conversion.
    """
    landmarks = hand_landmarks.landmark
    
    # Get normalized coordinates for Thumb Tip (4) and Index Tip (8)
    thumb_tip_lm = landmarks[TIP_IDS[0]]
    index_tip_lm = landmarks[TIP_IDS[1]]
    
    # Convert normalized coordinates to pixel coordinates
    thumb_x = int(thumb_tip_lm.x * w)
    thumb_y = int(thumb_tip_lm.y * h)
    
    index_x = int(index_tip_lm.x * w)
    index_y = int(index_tip_lm.y * h)
    
    # Calculate Euclidean distance between the two tips
    distance = np.sqrt((index_x - thumb_x)**2 + (index_y - thumb_y)**2)
    
    # Define a threshold for "pinch" (this value may need tuning based on camera/hand size)
    PINCH_THRESHOLD = 50 
    
    if distance < PINCH_THRESHOLD:
        return True, (index_x, index_y)
    else:
        return False, None


with mp_hands.Hands(
        static_image_mode=False,
        max_num_hands=1,
        min_detection_confidence=0.5,
        min_tracking_confidence=0.5) as hands:
    
    # --- Main Processing Loop ---
    while cap.isOpened():
        success, image = cap.read()
        if not success:
            continue
            
        h, w, c = image.shape # Get image dimensions for pixel conversion
        
        image.flags.writeable = False
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        results = hands.process(image)
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        # --- 3. Gesture Recognition and Display ---
        current_gesture_text = "No Hand Detected"

        if results.multi_hand_landmarks:
            for hand_landmarks in results.multi_hand_landmarks:
                
                # A. Finger Counting Logic
                fingers_up = check_fingers_up(hand_landmarks)
                count = sum(fingers_up)
                
                # B. Pinch Gesture Logic (Click Simulation)
                is_pinching, pinch_center = detect_pinch(hand_landmarks, h, w)
                
                # --- Update Display Text based on Gestures ---
                if is_pinching:
                    current_gesture_text = f"PINCH DETECTED! (Will 'Click')"
                    
                    # Highlight the pinch point
                    if pinch_center:
                        cv2.circle(image, pinch_center, 15, (0, 0, 255), -1) # Red Pinch Marker
                
                elif count > 0:
                    current_gesture_text = f"Number Guessing: {count} Fingers Up"
                    
                # Draw the standard hand landmarks
                mp_drawing.draw_landmarks(
                    image, hand_landmarks, mp_hands.HAND_CONNECTIONS)


        # --- 4. Display Status and Frame ---
        # Display the detected gesture text on the screen
        cv2.putText(image, current_gesture_text, (10, 30), 
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
        
        cv2.imshow('Hand Tracking Input System - Phase 2', image)
        
        if cv2.waitKey(5) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()